## Challenge A (Total: 3 Points):

#### Scenario 1: 
As a business intelligence developer, I would like to &quot;look back in time&quot; at previous data states of the ‘general_marketplaces.cln_listings’ table. While some source data
systems are built in a way that makes accessing historical data possible, this is not the case here: this
table has only current data state. In order to record changes to this mutable table over time, it is necessary to use a mechanism to do so.

In [ ]:
import os
import numpy as np
import pandas as pd
import pandas_gbq as pd_gbq
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from pandasql import sqldf
import datetime

In [2]:
### Create client using json file & project_id
client = bigquery.Client.from_service_account_json(json_credentials_path="credentials.json")

### Task 1 (1.5 Points): 
Create or describe a mechanism/script to register each data change (snapshot of
end of day) from the clean table ‘general_marketplaces.cln_listings’ into the fact table
‘general_marketplaces.fct_listings’ for each day of the interval considering validity of
the records (SCD type 2).

### Assumptions and Approach

##### Assumptions
- status_id 10 corresponds to an active listing
- status_id 15 corresponds to a listing deactivation/sale

##### fct_listings
- fct_listings is currently in a temporary state and needs to be updated according to new records in the cln_listings
- I've identitified the the following reasons to update/insert records in fct_listings:
    - New records (listing_id) in cln_listings which doesn't exist in fct_listings
    - Existing record (common listing_id & listing_date) with a new status_id
    
    

#### Merge Statement Approach
- Inserting / Updating data in my target table fct_listings
- Use the cln_listings table to update/insert records into fct_listings
- Limit cln_listings to only contain relevant records which aren't in fct_listings
    - New listing_id & listing_date_key
    - Existing listing_id & listing_date_key with new status_id
- When fct_listings record is updated (new status_id) we also last_updated_at field
- When new fct_listings record is inserted, both the last_updated_at & created_at field are set



#### Testing the merge statement
- The merge statement ran without error in BigQuery, but there we're no updates / rows inserted
- Additional testing possible by adding a row to cln_listings, running merge statement and checking fct_listings
    - Not done for time considerations

In [2]:
SQL_MERGE_STATEMENT = """
MERGE `smgmaxschlafli.fct_listings.fct_listings` 
USING (
  select distinct
    cln.listing_id
  , cln.price
  , cln.listing_date_key
  , cln.platform_id
  , cln.product_type_id
  , cln.status_id
  , cln.user_id
  , cln.creation_date
  , cln.last_update_date
  from `cln_listings.cln_listings` cln
  left join  `smgmaxschlafli.fct_listings.fct_listings`  fct on fct.listing_id = cln.listing_id and fct.listing_date_key = cln.listing_date_key
  where cln.status_id != fct.status_id
) AS staging
ON
      staging.listing_id = `smgmaxschlafli.fct_listings.fct_listings_clean`.listing_id  
  and staging.listing_date_key = `smgmaxschlafli.fct_listings.fct_listings_clean`.listing_date_key
WHEN MATCHED THEN
  UPDATE SET
      `smgmaxschlafli.fct_listings.fct_listings`.valid_to = staging.last_update_date
    , `smgmaxschlafli.fct_listings.fct_listings`.status_id = staging.status_id
    , `smgmaxschlafli.fct_listings.fct_listings`.last_updated_at = current_date()
WHEN NOT MATCHED BY TARGET THEN
  INSERT (
      listing_id
    , price
    , valid_from
    , valid_to
    , listing_date_key
    , platform_id
    , product_type_id
    , status_id
    , user_id
    , created_at
    , last_updated_at
    )
  VALUES (
      staging.listing_id
    , staging.price
    , staging.creation_date
    , staging.last_update_date
    , staging.listing_date_key
    , staging.platform_id
    , staging.product_type_id
    , staging.status_id
    , staging.user_id
    , current_date()
    , current_date()
    )
"""

#### Scenario 2:
As a business intelligence developer, I would like to see all the listings (classifieds) changes
on a daily basis for each platform in order to track its evolution and answer business questions.

### Task 2 (1.5 Points):
Create a report for the period from Dec. 2021 to Jan. 2022 (2 months). Also consider clean-up/mapping data changes that occurred during this period, for example related to
platform re-naming recorded in the dimensional table
‘general_marketplaces.dim_platform’.

### Assumptions and Approach


#### Assumptions
- Wasn't clear to me what'tracking changes' means
- Assumed that changes refers to the change in daily listings, split by platform

#### Approach
- Since data is split by Platform and we have different names for same platform (e.g. Tutti, Tutti.ch), I've cleaned the platform, removing '.ch'
- Daily aggregation of listings by platform, and also the day-over-day change through the use of window function (lag)


#### Output
- Given low number of records of 'cln_listings', the output data isn't very interesting
- Reverted inital plotting of data consequently

In [60]:
sql = """
    with daily_listings as (
        select
          replace(dp.platform,'.ch','') as platform
        , cl.creation_date
        , count(distinct listing_id) as listings
        from `cln_listings.cln_listings` cl
        left join `dim_platform.dim_platform` dp on cl.platform_id = dp.platform_id and cl.creation_date between dp.valid_from and dp.valid_to
        where cl.creation_date between '2021-12-01' and '2022-01-31'
        group by 1,2
    ) 
    select
      platform
    , creation_date
    , listings
    , lag(listings) over (partition by platform order by creation_date) as listings_previous_day
    , listings - lag(listings) over (partition by platform order by creation_date) as listings_change_dod
    from daily_listings
    order by 1,2 asc
"""

df = client.query(sql).to_dataframe()

In [61]:
df[df['platform']=='Anibis'].head(3)

,platform,creation_date,listings,listings_previous_day,listings_change_dod
0,Anibis,2021-12-01,2,<NA>,<NA>
1,Anibis,2021-12-03,1,2,-1
2,Anibis,2021-12-04,1,1,0


In [62]:
df[df['platform']=='Tutti'].head(3)

,platform,creation_date,listings,listings_previous_day,listings_change_dod
51,Tutti,2021-12-02,2,<NA>,<NA>
52,Tutti,2021-12-06,1,2,-1
53,Tutti,2021-12-07,1,1,0


In [63]:
df[df['platform']=='Ricardo'].head(3)

,platform,creation_date,listings,listings_previous_day,listings_change_dod
33,Ricardo,2021-12-02,1,<NA>,<NA>
34,Ricardo,2021-12-05,1,1,0
35,Ricardo,2021-12-10,1,1,0
